In [1]:
import edge_tts
import asyncio
import util    
import json
import sys
from IPython.display import Audio
from speakers.processors import load_preprocess,RvcProcessorData
from speakers.common.utils import get_abs_path
from omegaconf import OmegaConf

2023-08-13 22:59:52 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-08-13 22:59:53 | INFO | faiss.loader | Loading faiss with AVX2 support.
2023-08-13 22:59:53 | INFO | faiss.loader | Successfully loaded faiss with AVX2 support.


cuda:0
 Key with which the item will be registered {'device': 'cuda:0'}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60, 'x_max': 65}


/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_index" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
 

speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)
vits_processors,rvc_processors = load_preprocess(config=config.get('preprocess'))
 

{'preprocess': {'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}, 'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'rvc/model/hubert_base.pt', 'rvc_config_file': '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/rvc.yaml'}}}
{'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}


2023-08-13 23:00:15 | INFO | root | Loaded checkpoint '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/vits/model/G_953000.pth' (iteration 630)
2023-08-13 23:00:15 | INFO | fairseq.tasks.hubert_pretraining | current directory is /media/gpt4-pdf-chatbot-langchain/RVC-Speakers
2023-08-13 23:00:15 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2023-08-13 23:00:15 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_typ

Models loaded
{'name': 'rvc_speakers', 'hubert_model_path': 'rvc/model/hubert_base.pt', 'rvc_config_file': '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/rvc.yaml'}
/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/rmvpe.pt
Loading model: yiqing
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/yiqing/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: bob
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/bob/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: arianagrande
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/arianagrande/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: syz
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/syz/config.jso

In [85]:
from speakers.processors import VitsProcessorData

async def vits_tts_vc_func(
        vits_processors, rvc_processors,  
        input_text, model_index, pitch_adjust, f0_method, 
        feat_ratio, filter_radius, rms_mix_rate, resample_option,
        tts_speaker_index,speed: int, noise_scale: float,noise_scale_w:float
):
    if input_text is None:
        return (None, 'Please provide TTS text.')

    if tts_speaker_index is None:
        return (None, 'Please select TTS speaker.')

    if model_index is None:
        return (None, 'Please select a model.')
    print(input_text)
    data = VitsProcessorData( 
        text = input_text, 
        language= vits_processors.lang.index('简体中文') , 
        speaker_id= tts_speaker_index, 
        noise_scale = noise_scale, 
        speed = speed, 
        noise_scale_w= noise_scale_w
    )   
    SAMPLE_RATE = 22050 
    output_audio = vits_processors(data)
    
    # save audio to disk
    write_wav(f"test.wav", SAMPLE_RATE, output_audio) 
    # 将 NumPy 数组转换为 Python 列表
    audio_samples_list = output_audio.tolist()
    data = RvcProcessorData( 
        sample_rate=SAMPLE_RATE, 
        audio_samples= audio_samples_list ,
        model_index= model_index,
        f0_up_key= pitch_adjust,
        f0_method= f0_method,
        index_rate= feat_ratio,
        filter_radius= filter_radius,
        rms_mix_rate= rms_mix_rate,
        resample_sr= resample_option 
    )
    
    return rvc_processors(data)


In [86]:

loaded_models=[
        '%s - %s' % (
            m['metadata'].get('source', 'Unknown'),
            m['metadata'].get('name')
        )
        for m in rvc_processors.loaded_models
]
loaded_models

['Bilibili - 一清清清',
 'ALL - Bob Sponge',
 'ALL - Ariana Grande',
 'ALL - Stefanie Sun',
 '露露 - 露露',
 '毛毛 - 毛毛']

In [87]:

model_index = loaded_models.index('毛毛 - 毛毛')
model_index

5

In [88]:
import pandas as pd
speakers_table = pd.DataFrame(vits_processors.speakers, columns=["speaker"])     
speakers_table


,speaker
0,特别周
1,无声铃鹿
2,东海帝皇（帝宝，帝王）
3,丸善斯基
4,富士奇迹
...,...
799,支支
800,周良
801,珠函
802,祝明


In [89]:
tts_speaker_index= vits_processors.speakers.index('周良')
tts_speaker_index

800

In [90]:

noise_scale = 1
speed = 1
data = VitsProcessorData( 
    text = "观众朋友大家好", 
    language= vits_processors.lang.index('简体中文') , 
    speaker_id= tts_speaker_index, 
    noise_scale = noise_scale, 
    speed = speed, 
    noise_scale_w= 1.0
)   
SAMPLE_RATE = 22050 
output_audio = vits_processors(data)



In [91]:
from scipy.io.wavfile import write as write_wav
 
# save audio to disk
write_wav(f"test.wav", SAMPLE_RATE, output_audio) 

# play text in notebook
Audio(output_audio, rate=SAMPLE_RATE)

In [132]:
value="""观众朋友大家好,下面为大家带来一首非常精彩的角决赛,双方选手都迅速调整了自己的站位，准备展开激烈的交战。 """  

#变调(整数, 半音数量, 升八度12降八度-12)
f0_up_key=-1
f0_method =["pm", "harvest", "crepe", "rmvpe"]

#检索特征占比
index_rate = 0.9
#>=3则使用对harvest音高识别的结果使用中值滤波，数值为滤波半径，使用可以削弱哑音
filter_radius=1
#输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络
rms_mix_rate=1
#后处理重采样至最终采样率，0为不进行重采样
resample_rate=0

#noise_scale_w: noise_scale_w(控制音素发音长度)
noise_scale_w=1.3
#noise_scale(控制感情变化程度)
noise_scale = 0.9
#length_scale(控制整体语速)
speed = 1

out_sr, output_audio = await vits_tts_vc_func(vits_processors,rvc_processors, 
                                              value,model_index,f0_up_key,f0_method[3],
                                              index_rate, filter_radius, rms_mix_rate, resample_rate,
                                              tts_speaker_index,speed,noise_scale,noise_scale_w)



观众朋友大家好,下面为大家带来一首非常精彩的角决赛,双方选手都迅速调整了自己的站位，准备展开激烈的交战。 
-1
npy: 0.023456811904907227s, f0: 0.10556292533874512s, infer: 0.14055514335632324s


In [133]:
from scipy.io.wavfile import write as write_wav

# save audio to disk
write_wav(f"test.wav", out_sr, output_audio) 

# play text in notebook
Audio(output_audio, rate=out_sr)